<a href="https://colab.research.google.com/github/ar851060/slides_generator/blob/main/public_slides.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!pip install python-pptx
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.5/471.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.8 MB/s eta 0:00:00


In [ ]:
import openai, json
from pptx import Presentation
from pptx.util import Inches, Pt, Cm
from pptx.dml.color import RGBColor
import time
from google.colab import files, drive
import re
import requests
from bs4 import BeautifulSoup
import numpy as np
from pptx.enum.text import PP_ALIGN
from PIL import Image, ImageEnhance
import pickle
from openai.embeddings_utils import cosine_similarity, get_embedding
import time

# User control

In [ ]:
money = 0
openai.api_key = ""
ppt_filename = ""
topic = ""
language = "繁體中文"
number_pages = 8
prs = Presentation("/content/new_template.pptx")
pic_resource = "irasutoya" # dalle, irasutoya, flaticon
# if using irasutoya, you need to choose the method you want to pick up the pictures
method = "classification" # classification, chat, random
ppt_filename = ""

# If you need to use irasutoya-classification method, then run this


In [ ]:
res = requests.get(f'https://www.irasutoya.com/p/mail.html')
soup = BeautifulSoup(res.text, "html.parser")
li_list = soup.find_all("a", dir = "ltr")
label_list = []
for li in li_list:
    label_list.append(li.getText())

EMBEDDING_MODEL = "text-embedding-ada-002"
label_embeddings = []

for label in label_list:
    embed = openai.Embedding.create(input = label, model=EMBEDDING_MODEL)
    label_embeddings.append(embed['data'][0]['embedding'])
    money += embed["usage"]["total_tokens"] / 1000 * 0.0001
    time.sleep(3)

print(money)

# picture generator

In [ ]:
def irasudoya_pics(slide, money, get_keywords_methods:str="classification", number_keywords:int=3, temperature:float=0):

    res = requests.get(f'https://www.irasutoya.com/p/mail.html')
    soup = BeautifulSoup(res.text, "html.parser")
    li_list = soup.find_all("a", dir = "ltr")
    label_list = []
    for li in li_list:
        label_list.append(li.getText())

    def get_keywords(text, money, number_keywords):
        drive.mount('/content/drive')

        EMBEDDING_MODEL = "text-embedding-ada-002"
        embed = openai.Embedding.create(input = text, model=EMBEDDING_MODEL)
        text_embeddings = embed['data'][0]['embedding']
        money += embed["usage"]["total_tokens"] / 1000 * 0.0001

        with open("/content/drive/MyDrive/label_embeddings", "rb") as fp:
            label_embeddings = pickle.load(fp)

        def label_score(text_embeddings, label_embeddings):
            score = []
            for label in label_embeddings:
                cos = cosine_similarity(text_embeddings, label)
                score.append(cos)
            return score

        score = label_score(text_embeddings,label_embeddings)
        score = score / np.sum(score)
        key_label = np.random.choice(label_list, number_keywords, replace=False, p=score)

        return key_label, money

    def get_keywords_chat(text, money, number_keywords, temperature):

        find_pic_json = """{
        "input_text": "[[QUERY]]",
        "output_format": "json",
        "json_structure": {
            "keywords":['{{keywords}}','{{keywords}}','{{keywords}}']
        }
        }"""

        question_text = f"""
        According the content of this page, pick up {number_keywords} the most relevant Japanese keywords, and sort from the most relevant to the least relevant. Format as json.
        """

        question = question_text+"""
        the structure of json should be 'keywords':['{{keywords}}','{{keywords}}','{{keywords}}']。The list of Japanese keywords is following:###
        """ + ",".join(label_list)

        prompt = find_pic_json.replace("[[QUERY]]",question)

        completion = openai.ChatCompletion.create(model = "gpt-3.5-turbo", temperature = temperature, messages =[
            {"role":"user","content":"Please create the content of next page slide"},
            {"role":"assistant", "content":f"Header:{slide['header']}, Content:{slide['content']}"},
            {"role":"user","content":prompt}
            ])

        try:
            key_label = json.loads(completion.choices[0].message.content)["keywords"]
        except:
            key_label = json.loads(completion.choices[0].message.content)["json_structure"]["keywords"]
        money = money + completion.usage.prompt_tokens * 0.0015 / 1000 + completion.usage.completion_tokens * 0.002 / 1000

        return key_label, money


    text = slide['content']
    if get_keywords_methods == "classification":
        keywords_pic, money = get_keywords(text, money, number_keywords)
    elif get_keywords_methods == "chat":
        keywords_pic, money = get_keywords_chat(text, money, number_keywords, temperature)
    elif get_keywords_methods == "random":
        keywords_pic = np.random.choice(label_list, size=number_keywords, replace=False)
    else:
        assert False, f"there are only two methods to get pictures: 'classification', 'compress', and 'chat'. {get_keywords_methods} is not including."

    print(keywords_pic)
    print(f"Spent:{money}")
    pic_name = None
    while pic_name == None:
        for i in range(3,0,-1):
            keywords_string = "+".join(keywords_pic[:i])
            link_of_search = f'https://www.irasutoya.com/search/label/{keywords_string}'
            res = requests.get(link_of_search)
            soup = BeautifulSoup(res.text, "html.parser")
            find_pics = soup.find_all('div', id='post', class_='box')
            if find_pics:
                first_search = find_pics[np.random.randint(len(find_pics))].find("a")["href"]
                res = requests.get(first_search)
                soup = BeautifulSoup(res.text, "html.parser")
                div = soup.find_all('div', class_='separator')
                div_len = len(div)
                for i in range(div_len):
                    if "img" in str(div[i]):
                        pic_link = div[i].find("img")["src"]
                        if "http" in pic_link:
                            img = requests.get(pic_link)
                        else:
                            img = requests.get("https:"+pic_link)

                        pic_name = re.findall("s[0-9]+/(.*g)$",pic_link)[0]
                        with open(f"{pic_name}", "wb") as file:
                            file.write(img.content)
                        time.sleep(2)

                        return pic_name, money
        keywords_pic = np.random.choice(label_list, size=number_keywords, replace=False)


In [ ]:
def dalle_pics(slide, money, number_keywords:int=3, temperature:float = 0.9, size_picture:int=256, prompt:str=None):

    assert str(size_picture) in ["256","512","1024"], f"Dall-e only support the size of picture in 256, 512, and 1024. {size_picture} is not supported."

    text = f"According to this page, help me to extract {number_keywords} english keywords which are most relevant, and sort from the most relevant to the least. Format as json."

    find_pic_json = """{
        "input_text": """ + text + """
        The json construct should be 'keywords':['{{keywords}}','{{keywords}}','{{keywords}}']”,
        "output_format": "json",
        "json_structure": {
            "keywords":['{{keywords}}','{{keywords}}','{{keywords}}']
        }
        }"""

    completion = openai.ChatCompletion.create(model = "gpt-3.5-turbo", temperature = temperature, messages =[
        {"role":"user","content":"Please create the content of next page slide"},
        {"role":"assistant", "content":f"Header:{slide['header']}, Content:{slide['content']}"},
        {"role":"user","content":find_pic_json}
        ])
    try:
        keywords_pic = json.loads(completion.choices[0].message.content)["keywords"]
    except:
        keywords_pic = json.loads(completion.choices[0].message.content)["json_structure"]["keywords"]
    money = money + completion.usage.prompt_tokens * 0.0015 / 1000 + completion.usage.completion_tokens * 0.002 / 1000
    print(f"Spent:{money}")

    keys = ",".join(keywords_pic)
    if prompt:
        prompt = prompt.format(keywords = keys)
        response = openai.Image.create(
        prompt=prompt,
        n=1,
        size=f"{size_picture}x{size_picture}"
        )
    else:
        response = openai.Image.create(
        prompt=f"{keys};svg colored icon with blank background",
        n=1,
        size=f"{size_picture}x{size_picture}"
        )

    time.sleep(2)
    image_url = response['data'][0]['url']
    money += 0.016
    print(f"Spent:{money}\n")
    img = requests.get(image_url)
    pic_name = "".join(keywords_pic)+".png"
    with open(f"{pic_name}", "wb") as file:
        file.write(img.content)
    return pic_name, money

In [ ]:
def flaticon_pics(slide, money, number_keywords:int=3, temperature:float=0.9, size_picture:int=256):
    def pics_flaticon(keywords, size_picture):
        for key in keywords:
            link_of_search = f'https://www.flaticon.com/search?word={key}'
            res = requests.get(link_of_search, headers={"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"})
            soup = BeautifulSoup(res.text, "html.parser")
            find_pics = soup.find_all('a', class_='view link-icon-detail')
            if find_pics and size_picture > 128:
                return find_pics[np.random.randint(len(find_pics))].find("img")["data-src"].replace("128",str(size_picture))
        return None

    assert str(size_picture) in ["128","256","512"], f"The picture in Flaticon only support the size of 128, 256, and 512, {size_picture} is not supported."

    text = f"""
    extract {number_keywords} the most relevant English keywords, and sort from the most relevant to the least relevant. Format as json.
    """

    find_pic_json = """{
        "input_text": "According to the content of this page,""" + text +"""
                The structure of json should be 'keywords':['{{keywords}}','{{keywords}}','{{keywords}}']",
        "output_format": "json",
        "json_structure": {
            "keywords":['{{keywords}}','{{keywords}}','{{keywords}}']
        }
        }"""

    while True:
        completion = openai.ChatCompletion.create(model = "gpt-3.5-turbo", temperature = temperature, messages =[
            {"role":"user","content":"Please create the content of next page slide"},
            {"role":"assistant", "content":f"Header:{slide['header']}, Content:{slide['content']}"},
            {"role":"user","content":find_pic_json}
            ])
        print(completion.choices[0].message.content)
        try:
            keywords_pic = json.loads(completion.choices[0].message.content)["keywords"]
        except:
            keywords_pic = json.loads(completion.choices[0].message.content)["json_structure"]["keywords"]
        money = money + completion.usage.prompt_tokens * 0.0015 / 1000 + completion.usage.completion_tokens * 0.002 / 1000
        print(f"Spent:{money}")
        pic_link = pics_flaticon(keywords_pic)
        if temp < 2:
            temp += 0.1
        if pic_link is not None:
            break
    img = requests.get(pic_link)
    pic_name = re.findall(f"{size_picture}/[0-9]+/(.*g)$",pic_link)[0]
    with open(f"{pic_name}", "wb") as file:
        file.write(img.content)

    return pic_name, money

# The full process

In [ ]:
print("Start")

completion = openai.ChatCompletion.create(model = "gpt-3.5-turbo",temperature = 0.9, messages = [
    {"role":"system","content":f"""
    Write for a curious audience of college students in a friendly, conversational tone. You are an expert on this topic and want to share your knowledge with the world.
    Please follow these instructions in all your responses:
    1. Only use {language} language;
    2. Use {language} text as much as possible;
    3. Translate any other languages ​​into {language} where possible.
    4. The outlines are Mutually Exclusive Collectively Exhaustive.
    """},
    {"role":"user","content":f"""Please write a series of {language} {number_pages}-paged infographs，and the topic is '{topic}'. These infographs will be put on instrgram.
    Every infograph including 60 to 80 vocabulary, and give each infograph a attractive title.
    DO NOT generate covers, tags, the desciption of pictures. Do NOT need reference."""}
    ])
response = completion.choices[0].message.content
money = money + completion.usage.prompt_tokens * 0.0015 / 1000 + completion.usage.completion_tokens * 0.002 / 1000
print(f"Spent:{money}\n")
print(response)
print("Outline Finished")

開始
已經花了:0.002842
標題：日本四季節慶介紹

資訊圖卡 1:
標題：春天的櫻花季
文字：日本的櫻花季是一年中最受歡迎的季節之一。從三月至四月，櫻花盛開，人們會參加賞花活動、野餐和櫻花祭典，以慶祝這個美麗的季節。

資訊圖卡 2:
標題：夏天的盆踊祭
文字：夏天，日本人會參加盆踊祭，這是一種夏季盛會。人們穿著傳統服裝，在戶外跳舞、玩遊戲和品嚐傳統美食。這是一個歡樂的活動，讓人們在炎熱的夏天放鬆身心。

資訊圖卡 3:
標題：秋天的紅葉季
文字：秋天，日本的山區會變得滿是美麗的紅葉。人們會去登山、賞紅葉和品嚐秋季限定的美食。紅葉季是一個獨特而迷人的季節，吸引著遊客和本地人。

資訊圖卡 4:
標題：冬天的雪祭
文字：冬天，北海道的札幌舉辦了著名的雪祭，展示各種雕刻和巨大的冰雕。遊客可以欣賞這些冰雕，參加冰壺比賽和享受傳統的溫泉浴。雪祭是一個充滿冬季氛圍和樂趣的活動。

資訊圖卡 5:
標題：清明祭
文字：日本的清明祭是一個重要的傳統活動，用來追悼和悼念祖先。人們會去祖墳、清理墓地和供奉食物和花朵。這是一個平靜而有意義的時刻，讓人們思考並與過去的連結。

資訊圖卡 6:
標題：七五三節
文字：七五三節是一個慶祝孩子們成長的節日。在這個節日，三歲、五歲和七歲的孩子會穿上傳統服裝，參拜神社，祈求健康和幸福。這是一個充滿家庭和愛的美好時刻。

資訊圖卡 7:
標題：新年的初詣
文字：新年，日本人會參加初詣儀式，去神社祈求新的一年的好運和健康。人們會買小福袋、吃傳統食物和觀賞節目，以迎接新的開始。

資訊圖卡 8:
標題：祇園祭
文字：每年七月，京都舉辦著名的祇園祭。這個祭典包括各種活動，如巨大的神轎遊行、表演和傳統的體育比賽。祇園祭是京都最重要的節日之一，吸引著來自世界各地的遊客。

希望這些資訊圖卡能為你帶來對日本四季節慶的更多了解！如果你還有其他問題或需要更多資訊，歡迎提問！
大綱完成


In [ ]:
query_json = """{
    "input_text": "[[QUERY]]",
    "output_format": "json",
    "json_structure": {
        "title":"{{title}}",
        "slides":"{{slides}}"
       }
    }"""

question = f"""
According to the outlines of infographs above, reformat a series of {language} {number_pages}-paged infographs as json format, and come up a attractive title for this series of inforgraphs.
These will be put on instragram.
The structure of json should be “slides”:“{{slides}}”,the format of title is 'title':{{title}}，and each infographs should have a {{header}},{{content}}.
header is the attractive title of that infographs, and it should be less than 8 vocabulary.
The content should be showed as paragraphs.
"""

prompt = query_json.replace("[[QUERY]]",question)

completion = openai.ChatCompletion.create(model = "gpt-3.5-turbo",temperature = 0.9, messages = [
    {"role":"system","content":f"Write for a curious audience of college students in a friendly, conversational tone. You are an expert on this topic and want to share your knowledge with the world."},
    {"role":"assistant","content":response},
    {"role":"user","content":prompt}
    ])
response = completion.choices[0].message.content
money = money + completion.usage.prompt_tokens * 0.0015 / 1000 + completion.usage.completion_tokens * 0.002 / 1000
print(f"Spent:{money}")
print("Content Finished")

已經花了:0.007221
內容完成


In [ ]:
r = json.loads(response)

prs.slides[0].shapes.title.text = r["title"]
prs.slides[0].shapes.title.text_frame.paragraphs[0].font.color.rgb = RGBColor(64,64,64)


slide_data = r["slides"]

for slide in slide_data:


    if pic_resource == "flaticon":
        pic_name, money = flaticon_pics(slide, money)
    elif pic_resource == "dalle":
        pic_name, money = dalle_pics(slide, money)
    elif pic_resource == "irasutoya":
        pic_name, money = irasudoya_pics(slide, money, get_keywords_methods = method)
    else:
        assert False, f"We can only support three kinds of picture sources: flaticon, dall-e, irasudoya. {pic_resource} is not one of them."

    time.sleep(2)

    img = Image.open(pic_name)
    img = np.array(img)
    if len(img.shape)>2:
        try:
            img[:, :, 3] = img[:, :, 3] * 0.2
        except:
            layer = np.ones((img.shape[0],img.shape[1],1))*0.2
            result = np.append(img, layer, axis=2)
        img = Image.fromarray(img)
        img.save(pic_name)



    slide_layout = prs.slide_layouts[1]
    new_slide = prs.slides.add_slide(slide_layout)
    pic = new_slide.shapes.add_picture(f"/content/{pic_name}", Cm(7.73), Cm(10.91), width = Cm(22.64), height = Cm(22.95))
    # pic = new_slide.shapes.add_picture(f"/content/{pic_name}", Cm(3.57), Cm(3.18), width = Cm(7.54), height = Cm(7.54))


    if slide['header']:
        title = new_slide.shapes.title
        title.text = slide['header']
        title.text_frame.paragraphs[0].font.size = Pt(100)
        title.text_frame.paragraphs[0].font.underline = True


    if slide['content']:
        shapes = new_slide.shapes
        body_shape = shapes.placeholders[1]
        tf = body_shape.text_frame
        tf.clear()
        tf.text = slide['content'].replace(["。","."],"\n")
        paragraph_num = len(tf.paragraphs)
        for i in range(paragraph_num):
            p = tf.paragraphs[i]
            p.level = 0
            p.font.size = Pt(40)
            p.font.bold = True
            p.line_spacing = Pt(60)
            p.font.color.rgb = RGBColor(0,0,0)
            p.alignment = PP_ALIGN.CENTER

    new_slide.shapes._spTree.insert(2, pic._element)


print(f"Spent:{money}")
prs.save(f"{ppt_filename}.pptx")
files.download(f"/content/{ppt_filename}.pptx")